In [1]:
#import dependencies
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, InputLayer, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.regularizers import l2

In [3]:
#create df of preprocessed data - game data from 2004-2024
df = pd.read_csv('nbaPreProcessed.csv')
df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,...,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,isPlayoffGame,winner_binary
0,2024-06-17,Boston Celtics,BOS,2,0.792079,80,21,106,28,39,...,35,18,4,4,13,20,Boston Celtics,2023-2024,True,0
1,2024-06-14,Dallas Mavericks,DAL,7,0.611650,63,40,122,34,27,...,31,18,2,5,13,19,Dallas Mavericks,2023-2024,True,0
2,2024-06-12,Dallas Mavericks,DAL,7,0.607843,62,40,99,31,20,...,36,26,4,6,9,19,Boston Celtics,2023-2024,True,1
3,2024-06-09,Boston Celtics,BOS,2,0.795918,78,20,105,25,29,...,43,21,5,3,15,17,Boston Celtics,2023-2024,True,0
4,2024-06-06,Boston Celtics,BOS,2,0.793814,77,20,107,37,26,...,43,9,8,1,11,16,Boston Celtics,2023-2024,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,1,104,21,19,...,47,24,9,5,13,28,Indiana Pacers,2004-2005,False,1
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,1,95,22,31,...,41,20,9,2,16,26,Philadelphia 76ers,2004-2005,False,1
25728,2004-11-02,Los Angeles Lakers,LAL,14,1.000000,1,0,89,24,26,...,48,17,10,8,16,21,Los Angeles Lakers,2004-2005,False,0
25729,2004-11-02,Detroit Pistons,DET,9,1.000000,1,0,87,19,18,...,36,8,4,5,16,24,Detroit Pistons,2004-2005,False,0


In [7]:
#populate dfIDtoStat - should be double the size of df
homedf = df[['date', 'homeTeamSubject_id', 'homeTeam_win%', 'homeTeam_wins', 'homeTeam_losses', 'homeTeam_points_total', 'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO', 'homeTeam_PF']]
homedf.columns = ['date', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_total', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']
awaydf = df[['date', 'awayTeamSubject_id', 'awayTeam_win%', 'awayTeam_wins', 'awayTeam_losses', 'awayTeam_points_total', 'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made', 'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF']]
awaydf.columns = ['date', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_total', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']

dfIDtoStat = pd.concat([homedf, awaydf], ignore_index=True)
dfIDtoStat

,date,team_id,team_win%,team_wins,team_losses,team_points_total,team_FG,team_FG_made,team_3P,team_3P_made,team_O_Reb,team_D_Reb,team_Total_Reb,team_Ast,team_Stl,team_Blk,team_TO,team_PF
0,2024-06-17,2,0.792079,80,21,106,0.426966,38,0.333333,13,15,36,51,25,9,2,7,15
1,2024-06-14,7,0.611650,63,40,122,0.505495,46,0.405405,15,13,39,52,21,7,2,8,17
2,2024-06-12,7,0.607843,62,40,99,0.441860,38,0.360000,9,7,36,43,15,5,1,8,17
3,2024-06-09,2,0.795918,78,20,105,0.452381,38,0.256410,10,10,31,41,29,10,5,10,15
4,2024-06-06,2,0.793814,77,20,107,0.475610,39,0.380952,16,10,37,47,23,6,9,12,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,2004-11-03,12,1.000000,1,0,109,0.462366,43,0.235294,4,11,36,47,24,9,5,13,28
51458,2004-11-03,23,1.000000,1,0,98,0.444444,36,0.375000,9,14,27,41,20,9,2,16,26
51459,2004-11-02,8,0.000000,0,1,78,0.340909,30,0.250000,3,16,32,48,17,10,8,16,21
51460,2004-11-02,11,0.000000,0,1,79,0.394366,28,0.461538,6,9,27,36,8,4,5,16,24


In [8]:
windowConditions = {
    'team_win%': False,
    'team_wins': False,
    'team_losses': False,
    'team_points_total': True,
    'team_points_q1': True,
    'team_points_q2': True,
    'team_points_q3': True,
    'team_points_q4': True,
    'team_FG': True,
    'team_FG_made': True,
    'team_3P': True,
    'team_3P_made': True,
    'team_O_Reb': True,
    'team_D_Reb': True,
    'team_Total_Reb': True,
    'team_Ast': True,
    'team_Stl': True,
    'team_Blk': True,
    'team_TO': True,
    'team_PF': True,
}

In [9]:
#generate the X Y matrices
def df_to_X_y(df, dfWindow, window_size):
    X = []
    y = []
    #each team must have played window_size games before data can be extracted
    for index, row in df.iterrows():
        homeTeam_id = row['homeTeamSubject_id']
        awayTeam_id = row['awayTeamSubject_id']
        date = row['date']
        #limit dfWindow to include rows that occured before date of row we are on
        dfWindow_before_date = dfWindow[dfWindow['date']< date]
        homeTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].shape[0]
        awayTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].shape[0]
        if homeTeam_occurrences > window_size and awayTeam_occurrences > window_size:
            homeTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            awayTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            #condtional for average or top entry - home Team
            true_cols = [col for col, val in windowConditions.items() if val and col in homeTeam_window.columns]
            false_cols = [col for col, val in windowConditions.items() if not val and col in homeTeam_window.columns]
            homeTeam_window = np.concatenate([homeTeam_window[false_cols].iloc[0].to_numpy(), homeTeam_window[true_cols].mean(axis=0).to_numpy()])
            #conditional for average or top entry - away Team
            true_cols = [col for col, val in windowConditions.items() if val and col in awayTeam_window.columns]
            false_cols = [col for col, val in windowConditions.items() if not val and col in awayTeam_window.columns]
            awayTeam_window = np.concatenate([awayTeam_window[false_cols].iloc[0].to_numpy(), awayTeam_window[true_cols].mean(axis=0).to_numpy()])
            combined_window = np.hstack((homeTeam_window, awayTeam_window))
            X.append(combined_window)
            y.append(row['winner_binary'])
    
    return np.array(X), np.array(y)

In [10]:
#X1.Shape is num data, window size, variables
X, y = df_to_X_y(df, dfIDtoStat, 5)
X.shape, y.shape

((25634, 32), (25634,))

In [11]:
#80-10-10 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((20507, 32), (20507,), (2563, 32), (2563,), (2564, 32), (2564,))

In [12]:
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [13]:
#early stopping if model does not improve performance for 50 epochs
earlystopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
#simple model fit
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[earlystopping])

Epoch 1/100
641/641 [==============================] - 2s 2ms/step - loss: 0.6512 - accuracy: 0.6169 - val_loss: 0.6312 - val_accuracy: 0.6625
Epoch 2/100
641/641 [==============================] - 1s 1ms/step - loss: 0.6363 - accuracy: 0.6374 - val_loss: 0.6237 - val_accuracy: 0.6516
Epoch 3/100
641/641 [==============================] - 1s 1ms/step - loss: 0.6320 - accuracy: 0.6464 - val_loss: 0.6185 - val_accuracy: 0.6621
Epoch 4/100
641/641 [==============================] - 1s 1ms/step - loss: 0.6306 - accuracy: 0.6467 - val_loss: 0.6449 - val_accuracy: 0.6305
Epoch 5/100
641/641 [==============================] - 1s 998us/step - loss: 0.6258 - accuracy: 0.6483 - val_loss: 0.6159 - val_accuracy: 0.6664
Epoch 6/100
641/641 [==============================] - 1s 1ms/step - loss: 0.6251 - accuracy: 0.6500 - val_loss: 0.6148 - val_accuracy: 0.6609
Epoch 7/100
641/641 [==============================] - 1s 952us/step - loss: 0.6234 - accuracy: 0.6511 - val_loss: 0.6251 - val_accuracy: 0.

In [15]:
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)
print(accuracy_score(y_test, y_pred_binary))

81/81 [==============================] - 0s 663us/step
0.6641965678627145
